In [17]:
class SimpleClass():
    
    def __init__(self, name):
        print("hello " + name)
        
    def yell(self):
        print("YELLING")

In [2]:
s = "world"

In [3]:
type(s)

str

In [9]:
x = SimpleClass("Igor")

hello


In [5]:
x

In [10]:
x.yell()

YELLING


In [15]:
class ExtendedClass(SimpleClass):
    
    def __init__(self):
        super().__init__()
        print("EXTEND")

In [16]:
y = ExtendedClass()

hello
EXTEND


In [14]:
y.yell()

YELLING
